In [59]:
# Convolutional Neural Networks - Cats vs Dogs: CNN model to classify cats vs dogs
# Shomik Jain, USC CAIS++

# References: Dataset -- https://www.kaggle.com/c/dogs-vs-cats/data

In [60]:
# Convolutional Neural Networks
# stack successive layers of convolution & max pooling
    # results in a low dimensional, meaningful signal related to certain abstract features detected
# decision network: fully connected layers -- make decision based on signal produced by conv layers

In [61]:
# Convolutional Layers: scan input for structured features
# convolution: mathematical operation used to extract features from an image
    # image kernal matrix: applied to image by sliding over the image 
      # and for each position, computing element wise multiplication and summing result
# CNNs learn values of kernal filters, stack multiple layers of feature detectors (kernals)
  # on top of eachother for abstracted levels of feature detection 

In [62]:
# Imports
import os
import numpy as np
from scipy.ndimage import imread
import cv2
import sklearn.utils

DATA_PATH = './data/train/'
TEST_PERCENT = 0.1
# amount of dataset to use (for training time)
SELECT_SUBSET_PERCENT = 1.0

# The cat and dog images are of variable size we have to resize them to all the same size.
RESIZE_WIDTH=64
RESIZE_HEIGHT=64

EPOCHS = 9

In [63]:
# 1. Load the Data

X = []
Y = []

files = os.listdir(DATA_PATH)
# Shuffle to select about an equal number of dog and cat images.
shuffled_files = sklearn.utils.shuffle(files)
select_count = int(len(shuffled_files) * SELECT_SUBSET_PERCENT)

print('Going to load %i files' % select_count)

subset_files_select = shuffled_files[:select_count]

DISPLAY_COUNT = 1000

for i, input_file in enumerate(subset_files_select):
    if i % DISPLAY_COUNT == 0 and i != 0:
        print('Have loaded %i samples' % i)
        
    img = imread(DATA_PATH + input_file)
    # Resize the images to be the same size.
    img = cv2.resize(img, (RESIZE_WIDTH, RESIZE_HEIGHT), interpolation=cv2.INTER_CUBIC)
    X.append(img)
    if 'cat' == input_file.split('.')[0]:
        Y.append(0.0)
    else:
        Y.append(1.0)
        
X = np.array(X)
Y = np.array(Y)

test_size = int(len(X) * TEST_PERCENT)

test_X = X[:test_size]
test_Y = Y[:test_size]
train_X = X[test_size:]
train_Y = Y[test_size:]

print('Train set has dimensionality %s' % str(train_X.shape))
print('Test set has dimensionality %s' % str(test_X.shape))

# Normalization
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
train_X /= 255
test_X /= 255

Going to load 25000 files


/Users/shomikj/anaconda3/envs/caispp/lib/python3.5/site-packages/ipykernel/__main__.py:21: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.


Have loaded 1000 samples
Have loaded 2000 samples
Have loaded 3000 samples
Have loaded 4000 samples
Have loaded 5000 samples
Have loaded 6000 samples
Have loaded 7000 samples
Have loaded 8000 samples
Have loaded 9000 samples
Have loaded 10000 samples
Have loaded 11000 samples
Have loaded 12000 samples
Have loaded 13000 samples
Have loaded 14000 samples
Have loaded 15000 samples
Have loaded 16000 samples
Have loaded 17000 samples
Have loaded 18000 samples
Have loaded 19000 samples
Have loaded 20000 samples
Have loaded 21000 samples
Have loaded 22000 samples
Have loaded 23000 samples
Have loaded 24000 samples
Train set has dimensionality (22500, 64, 64, 3)
Test set has dimensionality (2500, 64, 64, 3)


In [64]:
# 2. Defining the Network 

# References
# Convolution layers: https://keras.io/layers/convolutional/
# Batch norm layer: https://keras.io/layers/normalization/
# Layer initializers: https://keras.io/initializers/
# Dense layer: https://keras.io/layers/core/#dense
# Activation functions: https://keras.io/layers/core/#activation
# Regulizers:
    # https://keras.io/layers/core/#dropout
    # https://keras.io/regularizers/
    # https://keras.io/callbacks/#earlystopping
    # https://keras.io/constraints/

In [65]:
# Import necessary layers.
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Activation, MaxPooling2D, Dropout, Flatten, BatchNormalization
from keras import optimizers
from keras import losses

model = Sequential()

# stack successive layers of convolution and max pooling
# results in a low dimensional, meaningful signal related to certain abstract features detected
model.add(Conv2D(input_shape=(64, 64, 3), filters = 16, kernel_size = 4))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(BatchNormalization())
model.add(Conv2D(filters = 32, kernel_size = 4))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(BatchNormalization())
model.add(Conv2D(filters = 64, kernel_size = 4))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = 2))

model.add(Flatten())

# decision network: fully connected layers 
# make decision based on signal produced by convolution layers
model.add(Dense(128))
model.add(Activation("relu"))

model.add(Dropout(0.5))

model.add(Dense(units=1))
model.add(Activation('sigmoid'))

# Cost Function: measure 
optimizer = optimizers.RMSprop(lr=5e-4)
loss = 'binary_crossentropy'

model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

In [66]:
# Define the batch size
batch_size = 32

model.fit(train_X, train_Y, batch_size=batch_size, epochs=EPOCHS, validation_split=0.2, verbose=1, shuffle=True)

Train on 18000 samples, validate on 4500 samples
Epoch 1/9
18000/18000 [==============================] - 113s 6ms/step - loss: 0.6362 - acc: 0.6504 - val_loss: 0.5817 - val_acc: 0.6909
Epoch 2/9
18000/18000 [==============================] - 110s 6ms/step - loss: 0.5359 - acc: 0.7334 - val_loss: 0.5006 - val_acc: 0.7516
Epoch 3/9
18000/18000 [==============================] - 109s 6ms/step - loss: 0.4757 - acc: 0.7745 - val_loss: 0.4358 - val_acc: 0.7976
Epoch 4/9
18000/18000 [==============================] - 108s 6ms/step - loss: 0.4220 - acc: 0.8070 - val_loss: 0.4489 - val_acc: 0.7989
Epoch 5/9
18000/18000 [==============================] - 108s 6ms/step - loss: 0.3813 - acc: 0.8295 - val_loss: 0.4340 - val_acc: 0.8038
Epoch 6/9
18000/18000 [==============================] - 108s 6ms/step - loss: 0.3376 - acc: 0.8507 - val_loss: 0.4681 - val_acc: 0.7951
Epoch 7/9
18000/18000 [==============================] - 118s 7ms/step - loss: 0.2973 - acc: 0.8729 - val_loss: 0.3733 - val_acc:

In [67]:
# 3. Evaluate the Model

loss, acc = model.evaluate(test_X, test_Y, batch_size=batch_size, verbose=1)

print('')
print('Got %.2f%% accuracy' % (acc * 100.))

2500/2500 [==============================] - 5s 2ms/step

Got 81.32% accuracy
